The code is based on kaggle project:
https://www.kaggle.com/code/rajmehra03/flower-recognition-cnn-keras
Flower Recognition CNN Keras, RAJ MEHROTRA, 2019

In [ ]:
!git clone https://github.com/j-anne/ML_flower_recognition.git

Data Preparation

In [ ]:
# Getting file path
import os

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
 
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score
from sklearn.preprocessing import LabelEncoder

#preprocess.
from keras.preprocessing.image import ImageDataGenerator

#dl libraraies
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from tensorflow.keras.utils import to_categorical

# specifically for cnn
from keras.layers import Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
 
import tensorflow as tf
import random as rn

# specifically for manipulating zipped images and getting numpy arrays of pixel values of images.
import cv2                  
from tqdm import tqdm                
from random import shuffle  
from PIL import Image

**Data Preparation**

In [ ]:
train_images = []
train_labels = []

In [ ]:
def load_images(DIR):
    for flower in tqdm(os.listdir(DIR)):
      label = flower
      path = DIR +'/'+ flower

      for image in os.listdir(path):
        img = cv2.imread(path+'/'+image)
        img = np.array(img).astype('uint8')
        img = cv2.resize(img, (150, 150))

        train_images.append(img)
        train_labels.append(label)

load_images("/content/ML_flower_recognition/flowers")

In [ ]:
fig,ax=plt.subplots(2,3)
fig.set_size_inches(10,10)
for i in range(2): 
  for j in range (3):
      l=rn.randint(0,len(train_labels))
      ax[i,j].imshow(train_images[l])
      ax[i,j].set_title(train_labels[l])

plt.tight_layout()

**Builing the Model**

In [ ]:
# modelling starts using a CNN.
# Uses RELU and Softmax for activation function
# Building convolutional model
model = Sequential()

# Input Layer
model.add(Conv2D(32, kernel_size=(5,5), padding = 'Same',activation ='relu', input_shape = (150,150,3)))
model.add(MaxPooling2D(pool_size=(2,2)))

# Convolution Layer
model.add(Conv2D(64, kernel_size=(3,3), padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(96, kernel_size=(3,3), padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128, kernel_size=(3,3), padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# Fully connected layer
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.7))

model.add(Dense(1000))
model.add(Activation('relu'))
model.add(Dropout(0.8))

# Output Layer
model.add(Dense(len(train_images), activation = "softmax"))

In [ ]:
# One Hot Encoding
le=LabelEncoder()
encoded_label=le.fit_transform(train_labels)
encoded_label=to_categorical(encoded_label,len(encoded_label))

# using cv2 for normalization
normalize_data = []

for img in train_images:
  norm = cv2.normalize(img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
  normalize_data.append(norm)

normalize_data = np.array(normalize_data)



In [ ]:
print(f"Normalize data shape: {normalize_data.shape}")
print(f"Encoded label shape: {encoded_label.shape}")

In [ ]:
plt.imshow(normalize_data[28])

**Train and Test Datasets**

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(normalize_data,encoded_label,test_size=0.25,random_state=42)

In [ ]:
# To resolve randomness of the model result
np.random.seed(42)
rn.seed(42)
tf.random.set_seed(42)

In [ ]:
batch_size=60
epochs=100

from keras.callbacks import ReduceLROnPlateau
red_lr= ReduceLROnPlateau(monitor='val_acc',patience=3,verbose=1,factor=0.1)

**Data Optimization**

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        horizontal_flip=True)  # randomly flip images horizontally

datagen.fit(x_train)

**Visualization**

In [ ]:
#Creates our batch of one image
fig,ax=plt.subplots(2,3)
fig.set_size_inches(10,10)

for i in range(2): 
  for j in range (3):
      l=rn.randint(0,len(train_labels))
      ax[i,j].imshow(normalize_data[l])
      ax[i,j].set_title('Flower: '+train_labels[l])

plt.tight_layout()

**Fitting the Model**

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.01),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
History = model.fit(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (x_test,y_test),
                              verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size)

In [ ]:
plt.plot(History.history['loss'])
plt.plot(History.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

print('The loss value is: ', History.history['loss'][-1])
print('The val loss value is: ', History.history['val_loss'][-1])

In [ ]:
plt.plot(History.history['accuracy'])
plt.plot(History.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

print('The accuracy of the model is: ', int(History.history['accuracy'][-1] * 100), '%')
print('The val accuracy of the model is: ', int(History.history['val_accuracy'][-1] * 100), '%')

In [ ]:
# getting predictions on val set.
pred=model.predict(x_test)
pred_digits=np.argmax(pred,axis=1)

In [ ]:
# now storing some properly as well as misclassified indexes'.
i=0
prop_class=[]
mis_class=[]

for i in range(len(y_test)):
    if(np.argmax(y_test[i])==pred_digits[i]):
        prop_class.append(i)
#     if(len(prop_class)==8):
#         break

i=0
for i in range(len(y_test)):
    if(not np.argmax(y_test[i])==pred_digits[i]):
        mis_class.append(i)

In [ ]:
print(f"Count of proper predicted images: {len(prop_class)}")
print(f"Count of mis-predicted images: {len(mis_class)}")

In [ ]:
count=0
fig,ax=plt.subplots(4,2)
fig.set_size_inches(15,15)
for i in range (4):
    for j in range (2):
        ax[i,j].imshow(x_test[prop_class[count]])
        ax[i,j].set_title("Predicted Flower : "+str(le.inverse_transform([pred_digits[prop_class[count]]]))+"\n"+"Actual Flower : "+str(le.inverse_transform([np.argmax(y_test[prop_class[count]])])))
        plt.tight_layout()
        count+=1

In [ ]:
count=0
fig,ax=plt.subplots(4,2)
fig.set_size_inches(15,15)
for i in range(4):
    for j in range (2):
        ax[i,j].imshow(x_test[mis_class[count]])
        ax[i,j].set_title("Predicted Flower : "+str(le.inverse_transform([pred_digits[mis_class[count]]]))+"\n"+"Actual Flower : "+str(le.inverse_transform([np.argmax(y_test[mis_class[count]])])))
        plt.tight_layout()
        count+=1